In [1]:
import requests as req
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
urls = [
        "https://www.gouvernement.fr/partage/12491-institut-des-hautes-etudes-de-defense-nationale-ihedn",
        "https://www.gouvernement.fr/partage/12489-numerique-educatif-le-premier-ministre-dresse-le-bilan-des-enseignements-de-la-crise-et-presente-des",
        "https://www.gouvernement.fr/partage/12488-le-numerique-au-service-de-l-innovation-pedagogique-et-de-l-egalite-des-chances",
        "https://www.gouvernement.fr/partage/12483-resultats-de-l-appel-a-projets-transports-collectifs-en-site-propre-et-poles-d-echanges-multimodaux",
        "https://www.gouvernement.fr/partage/12485-resultats-de-l-appel-a-projets-transports-collectifs-en-site-propre-tourcoing",
        "https://www.gouvernement.fr/partage/12480-signature-d-un-contrat-de-securite-integree-entre-l-etat-et-les-villes-de-rennes-et-de-saint-jacques",
        "https://www.gouvernement.fr/partage/12478-discours-de-cloture-de-jean-castex-au-17eme-congres-de-regions-de-france-a-montpellier",
        "https://www.gouvernement.fr/partage/12474-discours-de-jean-castex-au-congres-hlm-de-l-ush-de-bordeaux",
        "https://www.gouvernement.fr/partage/12473-congres-hlm",
        "https://www.gouvernement.fr/partage/12470-plan-de-reduction-des-tensions-de-recrutement",
        "https://www.gouvernement.fr/partage/12467-3eme-comite-interministeriel-aux-ruralites",
        "https://www.gouvernement.fr/partage/12465-plfss-2022-l-etat-s-engage-pour-le-grand-age-et-l-autonomie",
        "https://www.gouvernement.fr/partage/12461-remise-du-rapport-de-la-commission-pour-la-relance-durable-de-la-construction-de-logements",
        "https://www.gouvernement.fr/partage/12458-experimentation-de-la-recentralisation-du-financement-du-rsa-en-seine-saint-denis",
        "https://www.gouvernement.fr/partage/12455-journees-europeennes-du-patrimoine-2021",
        "https://www.gouvernement.fr/partage/12448-500-000eme-maprimerenov-discours-du-premier-ministre-jean-castex-a-chillu-mazarin-le-16-septembre",
        "https://www.gouvernement.fr/partage/12446-rencontre-avec-les-acteurs-de-l-enseignement-des-langues-regionales",
        "https://www.gouvernement.fr/partage/12445-remise-du-rapport-d-enquete-administrative-concernant-edouard-levrault",
        "https://www.gouvernement.fr/partage/12443-session-pleniere-de-rentree-du-conseil-economique-social-et-environnemental",
        "https://www.gouvernement.fr/partage/12436-journees-nationales-de-france-urbaine-a-nantes",
        "https://www.gouvernement.fr/partage/12433-remise-du-rapport-pour-un-usage-responsable-et-acceptable-par-la-societe-des-technologies-de",
        "https://www.gouvernement.fr/partage/12428-1-an-de-france-relance",
        "https://www.gouvernement.fr/partage/12432-discours-du-premier-ministre-jean-castex-compte-rendu-du-conseil-des-ministres-et-du-seminaire",
        "https://www.gouvernement.fr/partage/12426-discours-du-premier-ministre-jean-castex-conseil-national-de-l-industrie",
        "https://www.gouvernement.fr/partage/12412-commemoration-de-l-attentat-contre-des-humanitaires-francais-au-niger",
        "https://www.gouvernement.fr/partage/12411-pass-sanitaire-pour-rester-ensemble-face-au-virus"
    ]


In [3]:
discours_ = {'titre': [], 'date': [], 'discours': []}
for url in urls:
    response = req.get(url)
    soup = BeautifulSoup(response.text,  'html.parser')
    discours = soup.find('div', attrs={'class': 'field-item even first last'}).text
    titre = soup.find('h1', attrs={'class': 'rf-mb-3w'}).text
    date = soup.find('span', attrs={'class': 'rf-b-contenu-date'}).text
    
    discours_['discours'].append(discours.replace('\n', ' '))
    discours_['titre'].append(titre.replace('\n', ' '))
    discours_['date'].append(date.replace('Publié le : ', ''))

In [4]:
data = pd.DataFrame(discours_)

In [5]:
data.to_csv('discours.csv')

In [6]:
df = pd.read_csv('discours.csv')

In [7]:
len(df)

26

In [8]:
from collections import defaultdict,  Counter
import random
import sys

In [13]:
def build_markov_model( train_test, state_size ):
    markov_model = defaultdict(Counter)
    for i in range(len(train_test) - state_size):
        this_state = train_test[i:i+state_size]
        next_state = train_test[i+state_size]
        markov_model[this_state][next_state] += 1
    return markov_model

In [14]:
def generate_speech(markov_model, speech_size=300):
    all_done = False
    speech = []
    while not all_done:
        this_state = random.choice(list(markov_model))
        speech += list(this_state)
        
        for i in range(speech_size):
            population = list(markov_model[this_state])
            weight = markov_model[this_state].values()
            if len(population) == 0:
                break
                
            next_state = random.choices( population, weight)
            speech.extend(next_state)
            
            if len(speech) >= speech_size:
                all_done = True
                break
            this_state = this_state[i:] + speech[-1]
    print(''.join(speech))
            

In [74]:
texte = ""
for text in df['discours']:
    texte += "{} ".format(text)
    
len(texte)

365409

In [75]:
# texte = "Je suis inarrêtable maintenant, car je suis un génie!"
# text = "aida agira aigri"
model = build_markov_model( texte, state_size=10 )

In [76]:
model

defaultdict(collections.Counter,
            {'Madame la ': Counter({'m': 13,
                      's': 1,
                      'p': 28,
                      'M': 6,
                      'P': 7,
                      'V': 1}),
             'adame la m': Counter({'i': 6, 'a': 10}),
             'dame la mi': Counter({'n': 6}),
             'ame la min': Counter({'i': 6}),
             'me la mini': Counter({'s': 6}),
             'e la minis': Counter({'t': 9}),
             ' la minist': Counter({'r': 14}),
             'la ministr': Counter({'e': 14}),
             'a ministre': Counter({',': 4, ' ': 10}),
             ' ministre,': Counter({' ': 12}),
             'ministre, ': Counter({'c': 1,
                      'a': 3,
                      'à': 1,
                      'i': 1,
                      'l': 2,
                      'S': 1,
                      'e': 1,
                      'd': 1,
                      'm': 1}),
             'inistre, c': Counter({'h': 2}),
  

In [77]:
import json
json.dump(model, open('./model.json', 'w'), indent=4, sort_keys=True)

In [78]:
import markovify

In [79]:
speech = '/n'.join(json.load(open('./model.json', 'r')))
model_2 = markovify.Text(texte, state_size=2)

In [80]:
model_2.make_sentence()

'Les permis de construire ensemble les choses s’effondrer.'